In [1]:
import time
import urllib
from bs4 import BeautifulSoup
import html

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
client_id = 'YOJNMIRO64OhCZjBIWEO'
client_secret = 'ZHs9hqW8b9'

In [ ]:
keywords = ['네이버', '카카오', '라인', '쿠팡', '배달의민족', '당근마켓', '토스', '직방', '야놀자', '삼성전자', 'SK하이닉스', 'DB하이텍']

driver = webdriver.Chrome('/Users/leehyounshu/Documents/2024NLP/project/chromedriver-mac-x64/chromedriver')

for keyword in keywords: 
    base_url = 'https://openapi.naver.com/v1/search/news.json'
    encQuery = urllib.parse.quote(keyword) # 검색어
    n_display = 100 # 한 번에 표시할 검색 결과 개수
    sort = 'sim' # 검색 결과 정렬 방법: 정확도순으로 내림차순 정렬
    
    result_list = [] # 검색 결과 저장을 위한 빈 리스트 생성
    for start in range(1, 1000, 100): 
        # API 요청 URL 생성
        url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'

        my_request = urllib.request.Request(url) 
        my_request.add_header("X-Naver-Client-Id",client_id)
        my_request.add_header("X-Naver-Client-Secret",client_secret)

        response = urllib.request.urlopen(my_request) # API 요청 보내고 응답 받기
        rescode = response.getcode() # 응답 코드 확인
        
        if(rescode==200): # 응답 코드가 성공 인지 확인
            response_body = response.read() # 응답 본문 읽기
        else:
            print("Error Code:" + rescode)
            
        search_results = response_body.decode('utf-8') # 응답 본문을 문자열로 디코딩
        search_results = eval(search_results) # 문자열을 딕셔너리 형태로 변환
        
        result_list.append(search_results) # 검색 결과를 리스트에 추가
    
    
    news_links = [] # 뉴스 링크를 저장할 리스트 생성
    for result in result_list:
        for item in (result['items']):
            # 링크 추출 후 HTML 엔티티를 디코딩하고 역슬래시를 제거
            link = html.unescape(item['link']).replace('\\', '')
            if 'n.news.naver.com' in link: # 네이버 뉴스 링크인지 확인
                news_links.append(link) # 네이버 뉴스 링크를 리스트에 추가
        
    all_results = dict() # 모든 결과를 저장할 딕셔너리 생성

    for idx in tqdm(range(len(news_links))):
        link = news_links[idx] # 현재 인덱스의 링크 추출
        all_results[idx] = dict() # 인덱스를 키로 사용하여 딕셔너리 초기화
        driver.get(link) # 웹 드라이버로 해당 링크 열기
        time.sleep(1) # 페이지 로딩을 위해 1초 대기
        
        try: # 제목 요소 찾기
            title = driver.find_element(By.CLASS_NAME, 'media_end_head_headline')
        except NoSuchElementException: # 요소를 찾을 수 없으면 다음 링크로 넘어감
            continue
        
        title = title.text # 제목 텍스트 추출
        
        body = driver.find_element(By.ID, 'newsct_article') # 본문 요소 찾기
        body = body.text.replace('\n', '') # 본문 텍스트 추출 후 줄바꿈 문자 제거

        # 결과 딕셔너리에 URL, 제목, 본문 추가
        all_results[idx]['url'] = link
        all_results[idx]['title'] = title
        all_results[idx]['content'] = body

    df = pd.DataFrame(all_results).T # 딕셔너리를 데이터프레임으로 변환 후 전치
    
    # 데이터프레임을 CSV 파일로 저장
    df.to_csv('csv/{}_뉴스.csv'.format(keyword), encoding='utf-8-sig', index=False)

In [ ]:
keywords = ['네이버', '카카오', '라인', '쿠팡', '배달의민족', '당근마켓', '토스', '직방', '야놀자', '삼성전자', 'SK하이닉스', 'DB하이텍']

driver = webdriver.Chrome('/Users/leehyounshu/Documents/2024NLP/project/chromedriver-mac-x64/chromedriver')

for keyword in keywords: 
    base_url = 'https://openapi.naver.com/v1/search/blog.json'
    encQuery = urllib.parse.quote(keyword)
    n_display = 100
    sort = 'sim'
    
    result_list = []
    for start in range(1, 1000, 100):
        url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'

        my_request = urllib.request.Request(url)
        my_request.add_header("X-Naver-Client-Id",client_id)
        my_request.add_header("X-Naver-Client-Secret",client_secret)

        response = urllib.request.urlopen(my_request)

        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
        else:
            print("Error Code:" + rescode)
            
        search_results = response_body.decode('utf-8')
        search_results = eval(search_results)
        result_list.append(search_results)
    
    blog_links = [] # 블로그 링크를 저장할 리스트 생성
    for result in result_list:
        for item in (result['items']):
            # 링크 추출 후 HTML 엔티티를 디코딩하고 역슬래시를 제거
            link = html.unescape(item['link']).replace('\\', '')
            if 'blog.naver.com' in link: # 네이버 블로그 링크인지 확인
                blog_links.append(link) # 네이버 블로그 링크를 리스트에 추가
        
    all_results = dict() # 모든 결과를 저장할 딕셔너리 생성

    for idx in tqdm(range(len(blog_links))):
        link = blog_links[idx] # 현재 인덱스의 링크 추출
        driver.get(link) # 웹 드라이버로 해당 링크 열기
        time.sleep(1) # 페이지 로딩을 위해 1초 대기
        
        # 웹 드라이버를 해당 iframe으로 전환
        iframe = driver.find_element('id', 'mainFrame')
        driver.switch_to.frame(iframe)
        
        try: # 제목 요소 찾기
            title = driver.find_element(By.CLASS_NAME, 'pcol1')
        except NoSuchElementException: # 요소를 찾을 수 없으면 다음 링크로 넘어감
            continue        
        title = title.text
        
        try: # 본문 요소 찾기
            body = driver.find_element(By.CLASS_NAME, 'se-main-container')
        except NoSuchElementException: # 요소를 찾을 수 없으면 다음 링크로 넘어감
            continue 
        body = body.text.replace('\n', '') # 본문 텍스트 추출 후 줄바꿈 문자 제거
        
        all_results[idx] = dict() # 인덱스를 키로 사용하여 딕셔너리 초기화
        # 결과 딕셔너리에 URL, 제목, 본문 추가
        all_results[idx]['url'] = link
        all_results[idx]['title'] = title
        all_results[idx]['content'] = body

    df = pd.DataFrame(all_results).T
    
    df.to_csv('csv/{}_블로그.csv'.format(keyword), encoding='utf-8-sig', index=False)

In [2]:
# 데이터프레임을 저장할 리스트 생성
dfs = []

# CSV 파일 목록
files = ['네이버_뉴스.csv', '카카오_뉴스.csv', '라인_뉴스.csv', '쿠팡_뉴스.csv', '배달의민족_뉴스.csv', '당근마켓_뉴스.csv','토스_뉴스.csv', '직방_뉴스.csv', '야놀자_뉴스.csv', '삼성전자_뉴스.csv', 'SK하이닉스_뉴스.csv', 'DB하이텍_뉴스.csv',
         '네이버_블로그.csv', '카카오_블로그.csv', '라인_블로그.csv', '쿠팡_블로그.csv', '배달의민족_블로그.csv', '당근마켓_블로그.csv', '토스_블로그.csv', '직방_블로그.csv', '야놀자_블로그.csv', '삼성전자_블로그.csv', 'SK하이닉스_블로그.csv', 'DB하이텍_블로그.csv']

for file in files:
    try: # UTF-8 인코딩으로 CSV 파일 읽기
        df = pd.read_csv('csv/'+file, encoding='utf-8')
    except UnicodeDecodeError:
        try: # UTF-8 인코딩이 실패하면 CP949 인코딩으로 읽기 시도
            df = pd.read_csv('csv/'+file, encoding='cp949')
        except UnicodeDecodeError: # CP949 인코딩이 실패하면 EUC-KR 인코딩으로 읽기 시도
            df = pd.read_csv('csv/'+file, encoding='euc-kr')

    keyword = file.split(".")[0] # 파일명에서 키워드 추출 (확장자 제외)

    df["keyword"] = keyword # 데이터프레임에 'keyword' 열 추가
    dfs.append(df) # 데이터프레임 리스트에 추가

df_combined = pd.concat(dfs, ignore_index=True) # 모든 데이터프레임을 하나로 결합, 인덱스 무시

# 'content' 열에서 중복된 행 제거, 처음 등장하는 행 유지
df_combined.drop_duplicates(['content'], keep='first', ignore_index=True, inplace=True)
# 'content' 열에서 앞쪽 공백 제거
df_combined["content"] = df_combined["content"].str.replace("^ +", "")
# 'content' 열에서 빈 문자열을 NaN으로 대체
df_combined["content"] = df_combined["content"].replace("", np.nan) 
# NaN이 포함된 행 제거
df_combined = df_combined.dropna(how="any")
# 인덱스 재설정, 이전 인덱스 삭제
df_combined = df_combined.reset_index(drop=True)

print('총 문서 수: ', len(df_combined)) # 총 문서 수 출력

총 문서 수:  16312


In [204]:
df_combined.head()

,url,title,content,keyword
0,https://n.news.naver.com/mnews/article/366/000...,"[전문] 민희진 “네이버·두나무, 투자 무관한 사적 만남”",민 대표 “하이브가 제시한 증거 자료 모두 불법”하이브 “증거 짜깁기한 적 없어”…...,네이버_뉴스
1,https://n.news.naver.com/mnews/article/015/000...,"""네이버·두나무와의 만남은…"" 민희진 첫 입장 표명","민희진 어도어 대표, 기자회견 후 첫 입장""네이버, 두나무, 하이브와 4자 대면 하...",네이버_뉴스
2,https://n.news.naver.com/mnews/article/011/000...,"[전문] 기자회견 후 첫 입장 발표…민희진 “네이버·두나무, 투자 무관한 사적 만남""",서울경제스타DB[서울경제]민희진 어도어 대표가 경영권 확보를 위해 네이버와 업비트 ...,네이버_뉴스
3,https://n.news.naver.com/mnews/article/079/000...,"민희진 ""네이버·두나무 인수 제안 NO…뉴진스와 더 돈독""","민희진 어도어 대표 19일 공식 입장문 발표""하이브 제시 증거 모두 불법 취득""민희...",네이버_뉴스
4,https://n.news.naver.com/mnews/article/081/000...,민희진 “네이버·두나무 만남은 사적 자리…카톡 내용도 짜집기”,"민희진, 기자회견 후 첫 본인명의 입장 표명발언하는 민희진 어도어 대표 - 민희진 ...",네이버_뉴스


In [ ]:
keywords = ['네이버', '카카오', '라인', '쿠팡', '배달의민족', '당근마켓', '토스', '직방', '야놀자', '삼성전자', 'SK하이닉스', 'DB하이텍']

for keyword in keywords:
    filtered_df = pd.DataFrame() # 빈 데이터프레임 생성

    # 'keyword' 열에서 현재 키워드를 포함하는 행을 필터링
    filtered_rows = df_combined[df_combined['keyword'].str.contains(keyword, na=False)]   
    # 필터링된 행들을 빈 데이터프레임에 추가
    filtered_df = pd.concat([filtered_df, filtered_rows], ignore_index=True)
    # 'keyword' 열 삭제
    filtered_df.drop("keyword", axis=1, inplace=True)
    # 필터링된 데이터를 새로운 CSV 파일로 저장
    filtered_df.to_csv('csv_com/{}.csv'.format(keyword), encoding='utf-8-sig', index=False)